## Future Enhancements / To-do list

* Fix DURATION variable from static to dynamic based on specific file length
* Increase num_segments variable from 5 to 10+ in order to get samples at about 3 seconds each (needs dynamic duration to calculate)

In [27]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import os
import librosa
import math
import json

DATASET_PATH = "gen_file"
JSON_PATH = "predict_this_5.json"
SAMPLE_RATE = 22050

## NEED TO UPDATE TO BE DYNAMIC
DURATION = 30 # measured in seconds, but wont work for us in the long term
##

SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# identify the specific track duration
## frames, sample rate --> duration

# split the duration into 3 second segments
## send the number of segments into the save_mfcc function
## update the SAMPLES_PER_TRACK variable used in save_mfcc

def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    # build a dictionary to store data
    data = {
        "mapping": [], #"mapping": ["clacky","thocky"], # 0 = clacky, 1 = thocky
        "labels": [], # target outputs
        "mfcc": [] # training inputs
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # 1.2 -> 2
    
    # loop through all genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        print('\n dirpath: {},\n dirnames: {},\n filenames: {}'.format(dirpath,dirnames,filenames))
        
        # ensure that we are not at the root level
        if dirpath is dataset_path:
            # save the semantic
            dirpath_components = dirpath.split("/") # genre/blues ==> ["genre", "blues"]
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\n Processing {}".format(semantic_label))
            
            # process files for a specific genre
            for f in filenames:
                
                # load audio file
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                f_duration = librosa.get_duration(y=signal, sr=sr)
                
                # num segments is dependent on the duration / 3 second intervals
                new_num_segments = int(f_duration / 3)
                
                # process segments extracting mfcc and storing data
                
                #for s in range(new_num_segments): -- commenting out the dynamic segment piece
                for s in range(num_segments):    
                    # re-calculate based on specific file --commented out
                    #num_samples_per_segment = int((SAMPLE_RATE * f_duration) / new_num_segments)
                    
                    # reverting to static for now
                    num_samples_per_segment = int((SAMPLE_RATE * DURATION) / num_segments)
                    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length) # 1.2 -> 2
                    #
                    
                    start_sample = num_samples_per_segment * s 
                    finish_sample = start_sample + num_samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_fft=n_fft,
                                                n_mfcc=n_mfcc,
                                                hop_length=hop_length
                                               )
                    
                    mfcc = mfcc.T 
                    # store mfcc for segment only if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, s))                  
               
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=60) # num_segments back to being used


 dirpath: gen_file,
 dirnames: [],
 filenames: ['c L a C k Y    s P a C e B a R.wav']

 Processing gen_file
gen_file\c L a C k Y    s P a C e B a R.wav, segment:0
gen_file\c L a C k Y    s P a C e B a R.wav, segment:1
gen_file\c L a C k Y    s P a C e B a R.wav, segment:2
gen_file\c L a C k Y    s P a C e B a R.wav, segment:3
gen_file\c L a C k Y    s P a C e B a R.wav, segment:4
gen_file\c L a C k Y    s P a C e B a R.wav, segment:5
gen_file\c L a C k Y    s P a C e B a R.wav, segment:6
gen_file\c L a C k Y    s P a C e B a R.wav, segment:7
gen_file\c L a C k Y    s P a C e B a R.wav, segment:8
gen_file\c L a C k Y    s P a C e B a R.wav, segment:9
gen_file\c L a C k Y    s P a C e B a R.wav, segment:10


C:\Users\samys\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=0
  return f(*args, **kwargs)


In [ ]:
# predict sample
predict(model, X_to_predict, y_to_predict)